In [ ]:
pip install requests

In [ ]:
import requests

def get_product_data(barcode):
    url = f"https://world.openfoodfacts.org/api/v0/product/{barcode}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 1:
            product = data['product']
            return product
        else:
            return f"Product with barcode not found."
    else:
        return f"Error fetching data. Status code: {response.status_code}"

**Extract all columns in the dataset**

In [ ]:
# extract a product
barcode = "737628064502"
product_data = get_product_data(barcode)

# extract all columns
if isinstance(product_data, dict):
    for key, value in product_data.items():
        print(f"{key}: {value}")
else:
    print(product_data)

_id: 0737628064502
_keywords: ['and', 'asia', 'beverage', 'cereal', 'food', 'gluten', 'include', 'kit', 'kitchen', 'no', 'noodle', 'pasta', 'peanut', 'plant-based', 'potatoe', 'product', 'rice', 'seasoning', 'simply', 'stir-fry', 'thai', 'thailand', 'their', 'vegan', 'vegetarian']
added_countries_tags: []
additives_debug_tags: []
additives_n: 1
additives_old_n: 1
additives_old_tags: ['en:e330']
additives_original_tags: ['en:e330']
additives_prev_original_tags: ['en:e330']
additives_tags: ['en:e330']
allergens: en:peanuts
allergens_from_ingredients: en:peanuts, peanut
allergens_from_user: (en) en:peanuts
allergens_hierarchy: ['en:peanuts']
allergens_lc: en
allergens_tags: ['en:peanuts']
amino_acids_prev_tags: []
amino_acids_tags: []
brand_owner: Simply Asia Foods, Inc.
brand_owner_imported: Simply Asia Foods, Inc.
brands: Thai Kitchen,Simply Asia
brands_tags: ['thai-kitchen', 'simply-asia']
categories: Plant-based foods and beverages, Plant-based foods, Cereals and potatoes, Cereals and

In [ ]:
import requests
import pandas as pd

def get_first_n_product_details(n):
    base_url = "https://world.openfoodfacts.org/cgi/search.pl"
    product_details = []
    page = 1
    page_size = 100

    while len(product_details) < n:
        params = {
            'json': 1,
            'page_size': page_size,
            'page': page,
            'action': 'process'
        }
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            try:
                data = response.json()
                products = data.get('products', [])
                if not products:
                    break
                for product in products:
                    product_name = product.get('product_name', 'N/A')
                    allergens = product.get('allergens', 'N/A')
                    food_groups = product.get('food_groups','N/A'),
                    image_url = product.get('image_url','N/A')
                    product_details.append({
                        'Product Name': product_name,
                        'Allergens': allergens,
                        'Food Group': food_groups,
                        'Image':image_url
                    })
                    if len(product_details) >= n:
                        break
                page += 1
            except ValueError as e:
                print(f"JSON decoding error: {e}")
                break
        else:
            print(f"Error fetching data. Status code: {response.status_code}")
            break
    return product_details

# get the first 200 rows
first_200_product_details = get_first_n_product_details(200)

# convert to a pandas dataframe
df = pd.DataFrame(first_200_product_details)

**Sample Dataframe**

In [ ]:
pd.set_option('display.max_rows', 50)
df.head(50)

,Product Name,Allergens,Food Group,Image
0,Eau de source,,"(en:unsweetened-beverages,)",https://images.openfoodfacts.org/images/produc...
1,Prince Chocolat biscuits au blé complet,"en:eggs,en:gluten,en:milk,en:soybeans","(en:biscuits-and-cakes,)",https://images.openfoodfacts.org/images/produc...
2,Nutella,"en:milk,en:nuts,en:soybeans","(en:sweets,)",https://images.openfoodfacts.org/images/produc...
3,Sésame,"en:gluten,en:milk,en:sesame-seeds,fr:Avoine","(en:biscuits-and-cakes,)",https://images.openfoodfacts.org/images/produc...
4,Coca-cola,,"(en:sweetened-beverages,)",https://images.openfoodfacts.org/images/produc...
5,Nutella,"en:milk,en:nuts,en:soybeans","(en:sweets,)",https://images.openfoodfacts.org/images/produc...
6,Yeast Extract,"en:celery,en:gluten","(en:salty-and-fatty-products,)",https://images.openfoodfacts.org/images/produc...
7,Coca-Cola,,"(en:sweetened-beverages,)",https://images.openfoodfacts.org/images/produc...
8,Eau de source,,"(en:unsweetened-beverages,)",https://images.openfoodfacts.org/images/produc...
9,Coke zero,fr:Aspartame,"(en:artificially-sweetened-beverages,)",https://images.openfoodfacts.org/images/produc...


**Search for Products Images with allergens**

In [ ]:
import requests
from IPython.display import Image, display

# Function to search for products and return images
def get_product_images(query, num_images=10):
    url = f"https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        'search_terms': query,
        'json': 1,
        'page_size': num_images,
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()['products']
    else:
        return f"Error fetching data. Status code: {response.status_code}"

query = "fish"
products = get_product_images(query)

# display 10 images
for i, product in enumerate(products[:10]):
    image_url = product.get('image_url')
    product_name = product.get('product_name', 'N/A')
    allergens = product.get('allergens', 'N/A')

    print(f"Product Name: {product_name}")
    print(f"Allergens: {allergens}")

    if image_url:
        display(Image(url=image_url))
    else:
        print("No image available")
    print()

Product Name: 28 bâtonnets de surimi
Allergens: en:crustaceans,en:eggs,en:fish,en:gluten



Product Name: Bâton surimi
Allergens: en:crustaceans,en:eggs,en:fish,en:gluten



Product Name: BRANDADE DE MORUE
Allergens: en:fish



Product Name: Filets de maquereaux aux vin blanc et aux aromates
Allergens: en:fish



Product Name: Surimi saveur crabe
Allergens: en:crustaceans,en:fish,en:gluten



Product Name: Salade & Compagnie - Antibes
Allergens: en:eggs,en:fish,en:gluten,en:milk,en:soybeans,en:sulphur-dioxide-and-sulphites



Product Name: Foie de Morue nature au sel de Guérande 121g Cble
Allergens: en:fish,fr:Foie de morue



Product Name: Filets de Maquereaux Nature
Allergens: en:fish



Product Name: Filet de Saumon Purée de Brocolis
Allergens: en:fish,en:milk



Product Name: Fischstäbchen
Allergens: en:fish,en:gluten,de:Alaska-SeeIachsfiIet


**Search for Allergens**

In [ ]:
import requests
from IPython.display import Image, display

# Function to search for products containing a specific allergen and return images
def get_products_with_allergen(allergen, num_images=10):
    url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        'search_terms': allergen,
        'json': 1,
        'page_size': num_images,
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json().get('products', [])
    else:
        return f"Error fetching data. Status code: {response.status_code}"

allergen = "egg"
products = get_products_with_allergen(allergen)

# display 10 product images and allergen information
for i, product in enumerate(products[:10]):
    image_url = product.get('image_url')
    product_name = product.get('product_name', 'N/A')
    allergens = product.get('allergens', 'N/A')
    print(f"Product Name: {product_name}")
    print(f"Allergens: {allergens}")

    if image_url:
        display(Image(url=image_url))
    else:
        print("No image available")
    print()

Product Name: Brioche Tranchée Bio 400g
Allergens: en:eggs,en:gluten



Product Name: Real Mayonnaise
Allergens: en:eggs



Product Name: 4 Baguettes viennoises 340g
Allergens: en:eggs,en:gluten,en:milk



Product Name: Galettes au bon beurre bio
Allergens: en:eggs,en:gluten,en:milk



Product Name: Pain au lait bio x12 420g
Allergens: en:eggs,en:gluten,en:milk



Product Name: Seriously Good Times Mayonnaise
Allergens: 



Product Name: Bueno milk & hazelnut bars
Allergens: en:gluten,en:milk,en:soybeans



Product Name: Alpro Fermentiertes Kokosnussprodukt, Natur, Absolutely 350g
Allergens: 



Product Name: Queso blanco light cheese
Allergens: en:milk



Product Name: 10 Pains au lait frais sans sucres ajoutés 350g
Allergens: en:eggs,en:gluten,en:milk
